In [1]:
import japanize_matplotlib

# plot
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import yfinance as yf
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from stockstats import StockDataFrame

sns.set(font="IPAexGothic", rc={"figure.figsize": (11, 8)})
pd.options.display.float_format = "{:6.2f}".format

/usr/local/lib/python3.9/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
# Valid start and end: YYYY-MM-DD
# Valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
# Valid intervals: [1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo]
response = yf.download(
    tickers="SOXL",
    period="5y",
    interval="1d",
    group_by="ticker",
)

[*********************100%***********************]  1 of 1 completed


In [3]:
yfdata = response.copy().dropna()
yfdata = yfdata["1950-01":"202１-12"]  # 直近の暴落を除いて検証する
yfdata

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-05-22,6.01,6.12,5.99,6.11,6.00,5236500
2017-05-23,6.13,6.13,5.89,6.04,5.93,4482000
2017-05-24,6.14,6.19,6.07,6.15,6.05,3240000
2017-05-25,6.24,6.32,6.13,6.26,6.15,3831000
2017-05-26,6.26,6.34,6.12,6.33,6.22,4323000
...,...,...,...,...,...,...
2021-12-27,68.10,73.00,68.00,72.99,72.98,15598000
2021-12-28,74.07,74.07,69.57,70.52,70.51,14401800
2021-12-29,70.42,72.23,69.60,71.00,70.99,10702000


In [4]:
def convert_df_to_stock_df(df: pd.DataFrame) -> StockDataFrame:
    sdf = df.copy()
    sdf.rename(
        columns={
            "Open": "open",
            "High": "high",
            "Low": "low",
            "Close": "close",
            "Adj Close": "amount",
            "Volume": "volume",
        },
        inplace=True,
    )
    sdf.index.names = ["date"]
    return StockDataFrame(sdf)


def MACD(
    arr: pd.DataFrame, ema_short_period: int, ema_long_period: int, signal: int
) -> tuple[pd.Series, pd.Series]:
    sdf = convert_df_to_stock_df(arr)
    StockDataFrame.MACD_EMA_SHORT = 12
    StockDataFrame.MACD_EMA_LONG = 26
    StockDataFrame.MACD_EMA_SIGNAL = 9
    return (sdf["macd"], sdf["macds"])


def RSI(arr: pd.DataFrame, rsi: int) -> pd.Series:
    sdf = convert_df_to_stock_df(arr)
    return sdf["rsi_" + str(rsi)]

In [ ]:
class My_Strategy(Strategy):
    # macd
    prop_short = 12
    prop_long = 26
    prop_signal = 9

    # RSI
    prop_rsi = 14
    prop_sell_with_rsi = 70
    prop_buy_with_rsi = 30

    count = -1

    def init(self):
        self.macd, self.macd_signal = self.I(
            MACD, self.data.df, self.prop_short, self.prop_long, self.prop_signal
        )

        self.rsi = self.I(RSI, self.data.df, self.prop_rsi)

    def next(self):

        # 計算できていない場合トレードしない
        if (
            len(self.data.index) < self.prop_long
            or len(self.data.index) < self.prop_long
        ):
            return

        # 下記の条件の場合購入する
        # MACDがゴールデンクロスした。（ただしRSIの数値が高い場合は購入を見送る）
        # RSIが指定した数値を下回った。
        if (
            crossover(self.macd, self.macd_signal)
        ) or self.prop_buy_with_rsi > self.rsi[-1]:

            #             if len(self.trades) > 0:
            #                 return

            #             if crossover(self.macd, self.macd_signal):
            #                 print(str(self.data.index[-1]) + ": Buy with macd")

            #             if self.prop_buy_with_rsi > self.rsi[-1]:
            #                 print(str(self.data.index[-1]) + ": Buy with rsi")

            self.buy()

        elif (
            crossover(self.macd_signal, self.macd)
        ) or self.prop_sell_with_rsi < self.rsi[-1]:
            #             if len(self.trades) == 0:
            #                 return

            #             if crossover(self.macd_signal, self.macd):
            #                 print(str(self.data.index[-1]) + ": Sell with macd")

            #             if self.prop_sell_with_rsi < self.rsi[-1]:
            #                 print(str(self.data.index[-1]) + ": Sell with rsi")

            self.position.close()


bt = Backtest(yfdata, My_Strategy, cash=10000, commission=0.002, exclusive_orders=True)

# 最適化
optimize = bt.optimize(
    prop_rsi=range(10, 90, 5),
    prop_sell_with_rsi=range(10, 90, 5),
    prop_buy_with_rsi=range(10, 90, 5),
    method="grid",  # unuse model-based optimization
    constraint=lambda p: p.prop_buy_with_rsi < p.prop_sell_with_rsi,
    maximize="Equity Final [$]",
)

# 出力
# output = bt.run()
# print(output)

# print(output["_trades"])
# bt.plot()

/usr/local/lib/python3.9/site-packages/backtesting/backtesting.py:1488: UserWarning: Searching for best of 1920 configurations.
  output = _optimize_grid()
